Libraries

In [ ]:
import pandas as pd
import numpy as np

import torch
import cv2

from PIL import Image
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Checking device (GPU or CPU)

In [3]:
device=torch.device("cudo:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


Normalizing the image to make the training stable
- Min–Max Normalization → Convert pixel range 0–255 → [0,1]
- Z-score Normalization (Standardize using ImageNet mean/std)
- Mean Normalization (scale [-1,1])
- Per-Image Stardardization (Normalize each image, zero mean/unit std)
- Histogram Equalization → Convert image to grayscale and equalize histogram


In [ ]:
def minMax():
    transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    return transform

def zScore():
    transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform

def mean():
    transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    return transform

class perImageStandardization():
    def __call__(self, img):
        img=np.array(img).astype(np.float32)
        mean=img.mean()
        std=img.std()
        std_adj = max(std, 1.0/np.sqrt(img.size))  # avoid division by zero
        img = (img - mean) / std_adj
        img = torch.tensor(img).permute(2,0,1)  # HWC -> CHW
        return img

def perImageStd():
    transform=transforms.Compose([
        transforms.Resize((224,224)),
        perImageStandardization()
    ])
    return transform

class HistogramEqualization:
    def __call__(self, img):
        img = np.array(img.convert("L"))  # grayscale
        img_eq = cv2.equalizeHist(img)
        img_eq = torch.tensor(img_eq, dtype=torch.float32).unsqueeze(0) / 255.0  # CHW, scale [0,1]
        return img_eq

def histEqualization():
    return transforms.Compose([
        transforms.Resize((224,224)),
        HistogramEqualization()
    ])

Loading the Dataset
- Total number of data in the dataset: 27558
- Total number of infected cells: 13779
- Total number of unifected cells: 13779

In [ ]:
path = r"D:\cell_images"
transformDict = {
    "MinMax" : minMax(),
    "Z-Score" : zScore(),
    "Mean Normalization" : mean(),
    "Per-Image Std" : perImageStd(),
    "Histogram Equalization" : histEqualization()
}

for name, transform in transformDict.items():
    dataset=datasets.ImageFolder(root=path, transform=transform)